<a href="https://colab.research.google.com/github/ForestPearson/CS410-510-NLP-project/blob/Forest-Branch/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
import os
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import StringLookup

BATCH_SIZE = 64
BUFFER_SIZE = 10000
EPOCHS = 30
DIM = 256
RNN = 1024

#path = tf.keras.utils.get_file('input.txt', 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt')
path = tf.keras.utils.get_file('alls_well_that_ends_well.txt', 'https://raw.githubusercontent.com/ForestPearson/CS410-510-NLP-project/Forest-Branch/data/alls_well_that_ends_well.txt')

In [ ]:
text = open(path, 'rb').read().decode(encoding='utf-8')
print("Length:", len(text))
print(text[:500])

vocab = sorted(set(text))

Length: 129859
ACT I

SCENE I. Rousillon. The COUNT's palace.

Enter BERTRAM, the COUNTESS of Rousillon, HELENA, and LAFEU, all in black
COUNTESS
In delivering my son from me, I bury a second husband.
BERTRAM
And I in going, madam, weep o'er my father's death
anew: but I must attend his majesty's command, to
whom I am now in ward, evermore in subjection.
LAFEU
You shall find of the king a husband, madam; you,
sir, a father: he that so generally is at all times
good must of necessity hold his virtue to you; who


In [ ]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')

ids_from_chars = StringLookup(vocabulary=list(vocab), mask_token=None)
chars_from_ids = StringLookup(vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)
vocabSize = len(ids_from_chars.get_vocabulary())

ids = ids_from_chars(chars)
chars = chars_from_ids(ids)


In [ ]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [ ]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
#Convert to character indices
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

seq_length = 100
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

In [ ]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)
dataset = (dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [ ]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,return_sequences=True,return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [ ]:
model = MyModel(
    vocab_size=vocabSize,
    embedding_dim=DIM,
    rnn_units=RNN)

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 64) # (batch_size, sequence_length, vocab_size)


In [ ]:
model.compile(optimizer='adam', loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True))
# Directory where the checkpoints will be saved
dir = './data/epochs'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,save_weights_only=True)

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30
20/20 [==============================] - 1s 39ms/step - loss: 1.2210
Epoch 2/30
20/20 [==============================] - 1s 39ms/step - loss: 1.1043
Epoch 3/30
20/20 [==============================] - 1s 38ms/step - loss: 1.0448
Epoch 4/30
20/20 [==============================] - 1s 38ms/step - loss: 0.9923
Epoch 5/30
20/20 [==============================] - 1s 37ms/step - loss: 0.9351
Epoch 6/30
20/20 [==============================] - 1s 39ms/step - loss: 0.8720
Epoch 7/30
20/20 [==============================] - 1s 39ms/step - loss: 0.8087
Epoch 8/30
20/20 [==============================] - 1s 37ms/step - loss: 0.7383
Epoch 9/30
20/20 [==============================] - 1s 39ms/step - loss: 0.6701
Epoch 10/30
20/20 [==============================] - 1s 37ms/step - loss: 0.5990
Epoch 11/30
20/20 [==============================] - 1s 38ms/step - loss: 0.5270
Epoch 12/30
20/20 [==============================] - 1s 39ms/step - loss: 0.4601
Epoch 13/30
20/20 [==================

In [ ]:
class Generate(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  def predict(self, inputs, states=None):
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    predicted_logits, states = self.model(inputs=input_ids, states=states,return_state=True)
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    predicted_logits = predicted_logits + self.prediction_mask

    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    predicted_chars = self.chars_from_ids(predicted_ids)

    return predicted_chars, states

In [ ]:
Generator = Generate(model, chars_from_ids, ids_from_chars)
states = None
seed = tf.constant(['COUNTESS:'])
result = [seed]

for n in range(1000):
  seed, states = Generator.predict(seed, states=states)
  result.append(seed)

result = tf.strings.join(result)
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)

COUNTESS:
'To hundo in the boldness of his hald and sworn
Do not talke her; but you must not be mine, I will he do for heary;
Now his right be found in the enequare ten, was yours:
Clown
thy pathen's lack fortune fatch thy nose.
Her that will you say, so stand upon your virginity,
your old virginity, is like of loves, med compasitied
By the king in the nextry than nature
With him in gain, till she sighes; and be
sin, of our felt-so't.
HELENA
Ay, surely. dear shief, I ither thing to speak
our thousand ponsitious born brued him;
But give thyself unto kine some scores was her wind.
BERTRAM
Athendance, of a most charkle our pours to chisder.
Second Lord
Brow him for what procesting him to
resortion her his spurion, as
in heaven we couns to shadl but that which
I shall continue thankful.
Gentleman
I shall, my liege.
Exit

KING
Thus he his special nothing ever prologues.
Re-enter LAFEU, with like-
PAROLLES guarded, and First Soldier

BERTRAM
Ay, sir, deliver it fills.
DING
Thus young leature